In [ ]:
# 데이터셋 구조 : 제주방언 tab 표준어
# 파일 불러와서 제주방언끼리 / 표준어끼리 나눔 & test, train set 나눔 (여기서 나누나?흠)
# 각각 품사 태깅 작업 (표준어는 있는 라이브러리 쓰면 되고 _4가지 다? 일부만?)
# 제주어는 따로 작성한 사전 추가해서 태깅 작업
=> 토크나이징 된 문서 작성, json 형태로 저장
이 때 

# input data shape
https://blog.pingpong.us/tokenizer/

In [ ]:
효과적인 한국어 모델링을 위해서 tokenizer는 매우 중요하고
최신의 복잡한 모델을 사용하는 것보다 성능을 더 크게 올릴 수도 있는 요소입니다.

공백 기준의 tokenizer
형태소 분석기 기반 tokenizer (문제: 형태소 분석이 100% 확실하진 않음, 띄어쓰기 오류, 오타, 신조어에 취약,
                     속도가 느림, OOV 새로운 단어 몰랑)
Word Piece Model로 대표되는 Byte Pair Encoding (BPE) 기반의 tokenizer
(접두사, 접미사가 비교적 규칙적으로 잘 활용되는 영어의 경우 좋은 효과를 보임,
 한글의 경우 워낙 변형이 다양하기 때문에 잘 동작하지 않을 수도 있다)

In [ ]:
# tokenizer
형태소 분석기 기반 _ Mecab
BPE 기반 _ SentencePiece

# Making the imports

In [ ]:
import numpy as np
import os
import random
import tensorflow as tf
from coleections import Counter
import cvs

from keras.models import Model
from keras.layers import Input, LSTM, Dense

# Defining hyperparameters of the model

In [14]:
batch_size = 64
epochs = 100
latent_dim = 256
num_samples = 5000
data_path = 'jeju_final.txt'

In [19]:
# 데이터셋 나눠주기 (제주어 tab 표준어로 나뉘어져 있음)
input_texts = []
target_texts = []

with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[:min(num_samples, len(lines)-1)]:
    input_text, target_text = line.split('\t')
    target_text = '\t' + target_text.strip() + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)

In [ ]:
# 음절

In [2]:
# 띄어쓰기

In [12]:
# 형태소분석
# 제주어 방언으로 만든 파일 불러와서 사전 만들기 (한번만 하면 됨)
jeju_dictionary = dict()
with open('jeju_vocab.txt', encoding='utf-8-sig') as f:
    for line in f:
        line = line.split('\t')[0]
        jeju_dictionary[line] = len(jeju_dictionary)

jeju_reverse_dictionary = dict(zip(jeju_dictionary.values(), jeju_dictionary.keys()))

# 잘 되었나 확인
print(list(jeju_dictionary.items())[:10])
print(list(jeju_reverse_dictionary.items())[:10])
print('vocabulary size: ', len(jeju_reverse_dictionary))

[('가', 0), ('가게', 1), ('가게마씀', 2), ('가곡', 3), ('가그네', 4), ('가노라면', 5), ('가는', 6), ('가능', 7), ('가달', 8), ('가당', 9)]
[(0, '가'), (1, '가게'), (2, '가게마씀'), (3, '가곡'), (4, '가그네'), (5, '가노라면'), (6, '가는'), (7, '가능'), (8, '가달'), (9, '가당')]
vocabulary size:  1571


In [24]:
# 표준어 토큰화
# 품사 태깅
# mecab은 윈도우에서 안됨 (맥프로로 돌리장~ 결과는 json으로 저장해서 가져와서 쓰기)
from konlpy.tag import Kkma
kkma = Kkma()

#for inline in input_texts:
kkma.pos(input_texts[:10])

TypeError: No matching overloads found for kr.lucypark.kkma.KkmaInterface.morphAnalyzer(list), options are:
	public java.util.List kr.lucypark.kkma.KkmaInterface.morphAnalyzer(java.lang.String) throws java.lang.Exception

	at JPMethod::findOverload(native\common\jp_method.cpp:242)
	at JPMethod::findOverload(native\common\jp_method.cpp:245)
	at JPMethod::invoke(native\common\jp_method.cpp:253)
	at PyJPMethod::__call__(native\python\pyjp_method.cpp:142)


In [25]:
from konlpy.tag import Hannanum
hannanum = Hannanum()
hannanum.pos(input_texts[:10])

TypeError: No matching overloads found for kr.lucypark.jhannanum.comm.HannanumInterface.simplePos09(list), options are:
	public java.lang.String kr.lucypark.jhannanum.comm.HannanumInterface.simplePos09(java.lang.String)

	at JPMethod::findOverload(native\common\jp_method.cpp:242)
	at JPMethod::findOverload(native\common\jp_method.cpp:245)
	at JPMethod::invoke(native\common\jp_method.cpp:253)
	at PyJPMethod::__call__(native\python\pyjp_method.cpp:142)
